#### Reference

[Almost any Image Classification Problem using PyTorch](https://medium.com/@14prakash/almost-any-image-classification-problem-using-pytorch-i-am-in-love-with-pytorch-26c7aa979ec4)

[Plant Seedlings Classification](https://www.kaggle.com/c/plant-seedlings-classification)

Pytoch offers several pre-trained models and they can be easily loaded. Its main aim is to experiment faster using transfer learning on all available pre-trained models. 


**The following pre-trained models are available on PyTorch**

* resnet18, resnet34, resnet50, resnet101, resnet152

* squeezenet1_0, squeezenet1_1

* Alexnet

* inception_v3

* Densenet121, Densenet169, Densenet201

* Vgg11, vgg13, vgg16, vgg19, vgg11_bn. vgg13_bn, vgg16_bn, vgg19_bn

### The three cases in Transfer Learning 

1. Freezing all the layers except the final one

2. Freezing the first few layers

3. Fine-tuning the entire network.

All the models used above are written differently. Some use Sequential containers, which contain many layers and some directly contain just the layer. So it is important to check how these models are defined in PyTorch.

#### ResNet and Inception_V3

Since the Imagenet dataset has 1000 layers, We need to change the last layer as per our requirement. ***We can freeze whichever layer we don’t want to train and pass the remaining layer parameters to the optimizer.***

> 이미 만들어진 모델을 가져와서 쓰는데 그 모델 내부 layer에 대한 학습여부를 선택할 수 있나??

```python
if resnet:
    model_conv=torchvision.models.resnet50()
if inception:
  model_conv=torchvision.models.inception_v3()
## Change the last layer
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, n_class)
```

This model_conv has, In PyTorch there are children (containers) and each children has several childs (layers). Below is the example for resnet50,

```python
for name, child in model_conv.named_children():
    for name2, params in child.named_parameters():
        print(name, name2)
```

A long list of param are listed, some of them are shown below,
conv1 weight
```
bn1 weight
bn1 bias
....
fc weight
fc bias
```

Now if we want to freeze few layers before training, We can simple do using the following command:

```python
## Freezing all layers
for params in model_conv.parameters():
    params.requires_grad = False
## Freezing the first few layers. Here I am freezing the first 7 layers 
ct = 0
for name, child in model_conv.named_children():
    ct += 1
    if ct < 7:
        for name2, params in child.named_parameters():
        params.requires_grad = False
```

Changing the last layer to fit our new_data is a bit tricky and we need to carefully check how the underlying layers are represented. 

In [1]:
import torch, torchvision
import torch.nn as nn

In [2]:
def get_model(model_name):
    if model_name is 'resnet':
        model_conv = torchvision.models.resnet50()
    elif model_name is 'inception':
        model_conv = torchvision.models.inception_v3()
        
    return model_conv

In [3]:
model_conv = get_model('resnet')

In [4]:
n_class = 10
nb_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(nb_ftrs, n_class)

In [5]:
for name, child in model_conv.named_children():
    for name2, params in child.named_parameters():
        print(name, name2)

conv1 weight
bn1 weight
bn1 bias
layer1 0.conv1.weight
layer1 0.bn1.weight
layer1 0.bn1.bias
layer1 0.conv2.weight
layer1 0.bn2.weight
layer1 0.bn2.bias
layer1 0.conv3.weight
layer1 0.bn3.weight
layer1 0.bn3.bias
layer1 0.downsample.0.weight
layer1 0.downsample.1.weight
layer1 0.downsample.1.bias
layer1 1.conv1.weight
layer1 1.bn1.weight
layer1 1.bn1.bias
layer1 1.conv2.weight
layer1 1.bn2.weight
layer1 1.bn2.bias
layer1 1.conv3.weight
layer1 1.bn3.weight
layer1 1.bn3.bias
layer1 2.conv1.weight
layer1 2.bn1.weight
layer1 2.bn1.bias
layer1 2.conv2.weight
layer1 2.bn2.weight
layer1 2.bn2.bias
layer1 2.conv3.weight
layer1 2.bn3.weight
layer1 2.bn3.bias
layer2 0.conv1.weight
layer2 0.bn1.weight
layer2 0.bn1.bias
layer2 0.conv2.weight
layer2 0.bn2.weight
layer2 0.bn2.bias
layer2 0.conv3.weight
layer2 0.bn3.weight
layer2 0.bn3.bias
layer2 0.downsample.0.weight
layer2 0.downsample.1.weight
layer2 0.downsample.1.bias
layer2 1.conv1.weight
layer2 1.bn1.weight
layer2 1.bn1.bias
layer2 1.conv2.we

In [6]:
## Freezing all layers
for name, child in model_conv.named_children():
    for params in child.parameters():
#         print(params)
        params.requires_grad = False


In [7]:
    
## Freezing the first few layers. Here I am freezing the first 7 layers 
cnt = 0
for name, child in model_conv.named_children():
    cnt += 1
    if cnt < 7:
        for name2, params in child.named_parameters():
            params.requires_grad = False

#### Squeeze-Net

There are two variants of squeeze-net in PyTorch and Squeeze-net final layer is wrapped inside a container(Sequential). So we need to first list all the children layers inside it and convert the required layers according to our dataset and convert into back to a container and write it back to the class. 

```python
model_conv = torchvision.models.squeezenet1_1()
for name, params in model_conv.named_children():
    print(name)
```

features<br/>
classifier

```python
## How many In_channels are there for the conv layer
in_ftrs = model_conv.classifier[1].in_channels
## How many Out_channels are there for the conv layer
out_ftrs = model_conv.classifier[1].out_channels
## Converting a sequential layer to list of layers 
features = list(model_conv.classifier.children())
## Changing the conv layer to required dimension
features[1] = nn.Conv2d(in_ftrs, n_class, kernel_size,stride)
## Changing the pooling layer as per the architecture output
features[3] = nn.AvgPool2d(12, stride=1)
## Making a container to list all the layers
model_conv.classifier = nn.Sequential(*features)
## Mentioning the number of out_put classes
model_conv.num_classes = n_class
```

#### Dense-Net

It is very similar to Resnet but the last layer is named as classifier. 

```python
model_conv = torchvision.models.densenet121(pretrained='imagenet')
num_ftrs = model_conv.classifier.in_features
model_conv.classifier = nn.Linear(num_ftrs, n_class)
```


#### VGG and Alex-Net

It is similar to Squeeze-net. The last fc layers are wrapped inside a container, so we need to read that container and change the last fc layer as per our dataset requirements.

```python
model_conv = torchvision.models.vgg19(pretrained='imagenet')
# Number of filters in the bottleneck layer
num_ftrs = model_conv.classifier[6].in_features
# convert all the layers to list and remove the last one
features = list(model_conv.classifier.children())[:-1]
## Add the last layer based on the num of classes in our dataset
features.extend([nn.Linear(num_ftrs, n_class)])
## convert it into container and add it to our model class.
model_conv.classifier = nn.Sequential(*features)
```

We have seen how to freeze required layers and change the last layer for different networks. Now lets train the network using one of the nets. I am not going to mention this here in detail as it is already made available in my Github repo.
